## Overview

This tutorial demonstrates an end-to-end **Image Classification** pipeline using PyTorch and a **ResNet-50** deep learning model. The goal is to **classify Retinal Fundus Photographs** into different **device categories** based on their acquisition source. These images are labeled with metadata that includes the device type and diabetic severity level.

Key steps in the workflow include:

1. **Environment Setup**: Installing required libraries and getting access to dataset.

2. **Introduction to the problem**: Load and view example images from the dataset.

3. **Data Preparation**: Extracting image files, reading associated labels from a TSV file, and splitting the dataset into training, validation, and test sets.

4. **Custom Dataset Creation**: Defining a PyTorch Dataset class to load and transform images.  Applying normalization, and data augmentation techniques.  Loading data into memory for use.

5. **Model Building**: Loading a pre-defined ResNet-50 architecture and modifying its output layer for our custom classification task.  Set training parameters.

6. **Training & Validation**: Training the model using cross-entropy loss, tracking loss and accuracy over epochs, and visualizing performance.

7. **Prediction on Validation Data**: Evaluating model accuracy on validation data.

8. **Result Analysis**: Generating and displaying confusion matrices and plotting ROC curves for validation data.

9. **Try to Make the Model Perform Better**: Try a different optimizer and model initialization to improve performance.

10. **Chose a Model to Evaludate on Test Data**:  The test set should be reserved for when you have chosen a final model.  You cannot change your model after this point without creating a new test set.

11. **Explore the effect of hyperparameters in a sandbox.**

This tutorial is designed to help you understand the workflow of building a robust image classifier using real-world medical image data, along with visual performance evaluation.

## 1. Environment Setup: Importing Required Libraries
**Loading essential libraries for data processing, model training, and evaluation.**

In [ ]:
import os
# Set CUBLAS workspace configuration before any CUDA-related imports
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
from tqdm import tqdm
import time
start_total = time.time()
import glob
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
from torchvision.models import ResNet50_Weights

import pandas as pd

from PIL import Image

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

from itertools import cycle
from IPython.display import clear_output

torch.use_deterministic_algorithms(True)

## 2. Introduction to the problem: Load and view example images from the dataset.
**2a) Problem: Classify device from cropped, grayscale CFP images**

We have CFP images from 4 devices: iCare Eidon, Topcon Maestro2, Topcon Triton, and Optomed Aurora.  

**2b) We can load an image for one participant/eye and view.**

In [ ]:
img_eidon = np.asarray(Image.open('APPFL/examples/full/icare_eidon/1270_eidon_uwf_central_cfp_r_1.2.826.0.1.3680043.8.641.1.20240510.232115.43583.jpg'))
img_maestro = np.asarray(Image.open('APPFL/examples/full/topcon_maestro2/1270_maestro2_3d_macula_cfp_r_2.16.840.1.114517.10.5.1.4.907063120240510160806.2.1.jpg'))
img_triton = np.asarray(Image.open('APPFL/examples/full/topcon_triton/1274_triton_macula_12x12_cfp_r_2.16.840.1.114517.10.5.1.4.94005520240514143050.2.1.jpg'))
img_optomed = np.asarray(Image.open('APPFL/examples/full/optomed_aurora/1270_optomed_mac_or_disk_centered_cfp_r_2.25.2183161925995491838121778870991254610818.jpg'))

In [ ]:
plt.figure(figsize=(20,50))
plt.subplot(1,4,1)
plt.imshow(img_eidon)
plt.title('iCare Eidon')
plt.axis('off')

plt.subplot(1,4,2)
plt.imshow(img_maestro)
plt.title('Topcon Maestro2')
plt.axis('off')

plt.subplot(1,4,3)
plt.imshow(img_triton)
plt.title('Topcon Triton')
plt.axis('off')

plt.subplot(1,4,4)
plt.imshow(img_optomed)
plt.title('Optomed Aurora')
plt.axis('off')


**2c) Create a function to convert the full RGB images to cropped, grayscale images and convert the loaded images.**

In [ ]:
def rgb2gs(image):
    img_array = np.array(image)
    h, w, _ = img_array.shape
    top = (h - 224) // 2
    left = (w - 224) // 2
    center_crop = img_array[top:top+224, left:left+224]
    gray = np.dot(center_crop[...,:3], [0.2989, 0.5870, 0.1140])

    return gray

In [ ]:
img_eidon_gs = rgb2gs(img_eidon)
img_maestro_gs = rgb2gs(img_maestro)
img_triton_gs = rgb2gs(img_triton)
img_optomed_gs = rgb2gs(img_optomed)

**2d) Plot cropped, grayscale images.**

In [ ]:
plt.figure(figsize=(20,50))
plt.subplot(1,4,1)
plt.imshow(img_eidon_gs,cmap='gray')
plt.title('iCare Eidon')
plt.axis('off')

plt.subplot(1,4,2)
plt.imshow(img_maestro_gs,cmap='gray')
plt.title('Topcon Maestro2')
plt.axis('off')

plt.subplot(1,4,3)
plt.imshow(img_triton_gs,cmap='gray')
plt.title('Topcon Triton')
plt.axis('off')

plt.subplot(1,4,4)
plt.imshow(img_optomed_gs,cmap='gray')
plt.title('Optomed Aurora')
plt.axis('off')


**2e) Load all images for participant, convert to cropped grayscale and plot all the full RGB and cropped, grayscale images.**

In [ ]:
patient_id = '1270' #if you want to look at a different set of participant images change the "1270" to another number 1000-1300

#load image arrays and device name into a dictionary for plotting
img_dict = {}
for f in glob.glob('APPFL/examples/full/icare_eidon/' + patient_id + '*'):
    img_dict[f] = {}
    img_dict[f]['img'] = np.asarray(Image.open(f).resize((224, 224)))
    img_dict[f]['device'] = 'icare_eidon'
for f in glob.glob('APPFL/examples/full/topcon_maestro2/' + patient_id + '*'):
    img_dict[f] = {}
    img_dict[f]['img'] = np.asarray(Image.open(f).resize((224, 224)))
    img_dict[f]['device'] = 'topcon_maestro2'
for f in glob.glob('APPFL/examples/full/topcon_triton/' + patient_id + '*'):
    img_dict[f] = {}
    img_dict[f]['img'] = np.asarray(Image.open(f).resize((224, 224)))
    img_dict[f]['device'] = 'topcon_triton'
for f in glob.glob('APPFL/examples/full/optomed_aurora/' + patient_id + '*'):
    img_dict[f] = {}
    img_dict[f]['img'] = np.asarray(Image.open(f).resize((224, 224)))
    img_dict[f]['device'] = 'optomed_aurora'

In [ ]:
# get the keys of the dictionary and randomize
keys = list(img_dict.keys())
random.shuffle(keys)

#plot without titles
plt.figure(figsize=(10,10))
n=1
for k in keys:
    plt.subplot(5,5,n)
    plt.imshow(img_dict[k]['img'])
    plt.axis('off')
    n+=1

In [ ]:
plt.figure(figsize=(10,10))
n=1
for k in keys:
    plt.subplot(5,5,n)
    plt.imshow(img_dict[k]['img'])
    plt.title(img_dict[k]['device'])
    plt.axis('off')
    n+=1

In [ ]:
#load add cropped, grayscale image arrays and device name into a dictionary for plotting
for f in glob.glob('APPFL/examples/full/icare_eidon/' + patient_id + '*'):
    img_dict[f]['img_gs'] = rgb2gs(np.asarray(Image.open(f)))
for f in glob.glob('APPFL/examples/full/topcon_maestro2/' + patient_id + '*'):
    img_dict[f]['img_gs'] = rgb2gs(np.asarray(Image.open(f)))
for f in glob.glob('APPFL/examples/full/topcon_triton/' + patient_id + '*'):
    img_dict[f]['img_gs'] = rgb2gs(np.asarray(Image.open(f)))
for f in glob.glob('APPFL/examples/full/optomed_aurora/' + patient_id + '*'):
    img_dict[f]['img_gs'] = rgb2gs(np.asarray(Image.open(f)))
    

In [ ]:
plt.figure(figsize=(10,10))
n=1
for k in keys:
    plt.subplot(5,5,n)
    plt.imshow(img_dict[k]['img_gs'],cmap='gray')
    plt.axis('off')
    n+=1

In [ ]:
plt.figure(figsize=(10,10))
n=1
for k in keys:
    plt.subplot(5,5,n)
    plt.imshow(img_dict[k]['img_gs'],cmap='gray')
    plt.title(img_dict[k]['device'])
    plt.axis('off')
    n+=1

## 3. Data Preparation

**3a) Loading Dataset Labels**: We read the labels from a TSV file, which contains metadata about images.

In [ ]:
# importing data label
tsv_path = "APPFL/examples/cfp_images/labels.tsv"
df = pd.read_csv(tsv_path, sep='\t')

# printing top 5
df.head(5)

In [ ]:
#print the first 20 lines
print(df[['subject_id','device','partition']].head(20))

**3b) Splitting Dataset**: The dataset is split into training, validation, and test sets.

In [ ]:
# dividing data into diffrent dataframes based on train, validation and test
train_df = df[df["partition"] == "train"].copy()
val_df   = df[df["partition"] == "val"].copy()
test_df  = df[df["partition"] == "test"].copy()

print("Number of data sample \nTraining", len(train_df), "\nValidation", len(val_df), "\nTest", len(test_df))

In [ ]:
# Finding unique classes and giving it a number index
# in this case we have 4 classes
unique_classes = sorted(train_df["device"].unique())

# Mapping classes to a particular index
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(unique_classes)}
print("Class to index mapping:", class_to_idx)

train_df["label_idx"] = train_df["device"].map(class_to_idx)
val_df["label_idx"]   = val_df["device"].map(class_to_idx)
test_df["label_idx"]  = test_df["device"].map(class_to_idx)

num_classes = len(unique_classes)

**3c) Distribution of the Labels**:We can plot the distribution of the labels across the three partitions.

In [ ]:
train_labels = sorted(train_df["device"])
_ = plt.hist(train_labels)
plt.title('train distribution')
plt.show()

In [ ]:
val_labels = sorted(val_df["device"])
_ = plt.hist(val_labels)
plt.title('val distribution')
plt.show()

In [ ]:
test_labels = sorted(test_df["device"])
_ = plt.hist(test_labels)
plt.title('test distribution')
plt.show()

## 4. Defining the Custom Dataset Class

**4a) This class is used for loading images (cropping and converting to grayscale) and applying transformations.**

In [ ]:
# Defining a dataloader class, which returns image and its label
class AIREADIDataset_grayscalecrop(Dataset):
    def __init__(self, df, transform=None, preload=False):
        """
        Args:
          df: a DataFrame with at least ['file_path', 'label_idx'] columns
          transform: torchvision transforms (augmentations) to apply
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.preload = []
        if preload:
            partition = self.df["partition"][0]
            npsavfn = f"APPFL/examples/cfp_images/graycrop-{partition}.npy"
            if os.path.isfile(npsavfn):
                self.preload = np.load(npsavfn)
                print('loaded: ' + partition)
            else:
                for i in tqdm(range(0,len(self.df))):
                    row = self.df.iloc[i]
                    img_path = "APPFL/examples/cfp_images/" + row["file_path"]
                    image = Image.open(img_path).convert("RGB")
                    img_array = np.array(image)
                    #take only the 224x224 center square of the image
                    h, w, _ = img_array.shape
                    top = (h - 224) // 2
                    left = (w - 224) // 2
                    center_crop = img_array[top:top+224, left:left+224]
                    #convert to grayscale
                    gray = np.dot(center_crop[...,:3], [0.2989, 0.5870, 0.1140])
                    gray_3ch = np.stack((gray,)*3, axis=-1).astype(np.uint8)
                    self.preload.append(gray_3ch)
                self.preload = np.asarray(self.preload)
                np.save(npsavfn, self.preload)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        label = row["label_idx"]
        dm_severity = row["dm_severity"]
        
        if len(self.preload) > 0:
            image = Image.fromarray(self.preload[idx])
        else:
            img_path = "APPFL/examples/cfp_images/" + row["file_path"]
            # load the image
            image = Image.open(img_path).convert("RGB").resize((224, 224))

        # apply transforms
        if self.transform:
            image = self.transform(image)

        return image, label, dm_severity

**4b) Transformation of all images, normalization to ImageNet statistics  Augmenting training images to have horizontal flips and rotations for online augementation.**

In [ ]:
# Defining transformation of image
# Augmenting training data to have horizontal flips and rotations
train_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally
    T.RandomRotation(degrees=15),  # Randomly rotate images
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

val_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

**4c) Call the loader class with partition filepaths and transforms**

In [ ]:
# Loading datasets into memory for faster training
train_dataset = AIREADIDataset_grayscalecrop(train_df, train_transform, preload=True)
val_dataset   = AIREADIDataset_grayscalecrop(val_df, val_transform, preload=True)
test_dataset  = AIREADIDataset_grayscalecrop(test_df, val_transform, preload=True)

## 5. Model Building

**5a) Set batch size and create dataloader instances.**

In [ ]:
# Define batch size for data loading, batch sie is the number of training examples used during one iteration of model training
batch_size = 64
torch.set_num_threads(os.cpu_count())
num_workers = os.cpu_count()

# Create DataLoader instances for efficient batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))

**5b) Define device to use for training and inference.**

In [ ]:
# Define the device (GPU if available, else CPU), if this says CPU please raise your hand
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

**5c) Define methods used during model training, defines what to do each train step, validation inference, and plotting the learning curves.**

In [ ]:
# Methods to help with model training
def train_step(model, dataloader, optimizer, criterion, device, batch_losses):
    """Performs a single training step over the dataset."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(dataloader, desc="Training", leave=True)

    for batch_idx, (images, labels, _) in enumerate(progress_bar):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        batch_losses.append(loss.item())
        progress_bar.set_postfix(batch_loss=loss.item())

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    return epoch_loss, epoch_acc

def validate_step(model, dataloader, criterion, device):
    """Performs a single validation step over the dataset."""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(dataloader, desc="Validating", leave=True)

    with torch.no_grad():
        for batch_idx, (images, labels, _) in enumerate(progress_bar):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix(batch_loss=loss.item())

    avg_loss = total_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

def plot_training_progress(batch_losses, train_accuracies, val_accuracies, epoch):
    """Plots the training loss and accuracy over epochs."""
    plt.figure(figsize=(12, 5))

    # Loss Curve
    plt.subplot(1, 2, 1)
    plt.plot(batch_losses, label='Train Batch Loss', alpha=0.5, color='blue')
    plt.xlabel('Iterations (Batches & Epochs)')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    # Accuracy Curve
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epoch+2), train_accuracies, label='Train Accuracy', marker='o')
    plt.plot(range(1, epoch+2), val_accuracies, label='Val Accuracy', marker='o')
    plt.ylim([0.2, 1.05])
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training & Validation Accuracy')
    plt.legend()

    plt.show()

def train_model(model, train_loader, val_loader, optimizer, criterion, device, num_epochs):
    """Trains and validates the model over multiple epochs."""
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    batch_losses = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        train_loss, train_acc = train_step(model, train_loader, optimizer, criterion, device, batch_losses)
        train_accuracies.append(train_acc)

        val_loss, val_acc = validate_step(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        clear_output(wait=True)
        print(f"\nEpoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}\n")

        plot_training_progress(batch_losses, train_accuracies, val_accuracies, epoch)

    return train_accuracies, val_accuracies, val_losses

**5d) Defining model architecture: a pre-existing RESNET50 model architecture.**

In [ ]:
#this will make the initializtion the same each time it is run for the same setting of parameters, just run once
def set_seed(seed=42):
    random.seed(seed)                         # Python built-in random
    np.random.seed(seed)                      # NumPy
    torch.manual_seed(seed)                   # PyTorch CPU
    torch.cuda.manual_seed(seed)             # PyTorch CUDA (if using GPU)
    torch.cuda.manual_seed_all(seed)          # All CUDA devices (if using multi-GPU)

set_seed(12)
#Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Load the ResNet50 model
model = models.resnet50(weights=None)

# Modify the last layer for our classification task
# Changing to 4 classes as we have 4 devices to classify it into
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Transfer model to device (GPU or CPU)
model = model.to(device)

**5e) Choose a cost function, an optimizer and set learning rate.**

In [ ]:
# Define loss function, the function that we want to minimize to find the best assignment of labels 
criterion = nn.CrossEntropyLoss()  #CE quantifies how well a model's predicted probability distribution aligns with the true, or actual, probability distribution of the data

#set the optimier and learning rate (lr), there are different mathemactical methods to minimize loss functions, the learning rate dictates how big a step you take toward the minima
optimizer = optim.SGD(model.parameters(), lr = 1e-2, momentum=0.9) 

## 6. Running training: Call training function with desired number of epochs.

In [ ]:
#The number of eopch sets how many the model learns over the entire dataset in 
start = time.time()
train_model(model=model, train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            num_epochs=24)
print("Training time: ", time.time() - start)

## 7. Testing model: Perform inference with model on validation dataset

**7a) We are saving the test set for the final model chosen, so we evaluate on the validation set for now.**

In [ ]:
# --------------------- VAL ---------------------
all_preds = []
all_labels = []
all_dm_severity = []
print("Evaluating on val set...")
if len(val_df) > 0:
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels, dm_severity in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_dm_severity.extend(dm_severity)

    val_loss /= val_total
    val_acc = val_correct / val_total
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


    # Create a DataFrame to store results
df_results_val1 = pd.DataFrame({
    'true_device': all_labels,
    'pred_device': all_preds,
    'dm_severity': all_dm_severity
})


**7b) Saving test set results to a dataframe for use below.**

In [ ]:
# Create a DataFrame to store results
df_results_val1 = pd.DataFrame({
    'true_device': all_labels,
    'pred_device': all_preds,
    'dm_severity': all_dm_severity
})

## 8. Evaluate model performace on the Validation Set

**8a) Plot confusion matrix of device type for the val set.  This allows for interrogation of model performance on each class (device).**

In [ ]:
#define a function to plot a confusion matrix (cm)
def plot_confusion_matrix(cm, device_names, title="Confusion Matrix"):
    plt.figure()
    plt.imshow(cm, interpolation='nearest', aspect='auto')
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(device_names))
    plt.xticks(tick_marks, device_names, rotation=45, ha='right')
    plt.yticks(tick_marks, device_names)

    # Optionally annotate cells with counts
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i, j]),
                     horizontalalignment="center",color="white")
                     #color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.xlabel('Predicted Device')
    plt.ylabel('True Device')
    plt.show()

In [ ]:
# Generate confusion matrices with confusion_matrix function from scikit-learn
true_devs = df_results_val1['true_device'].values
pred_devs = df_results_val1['pred_device'].values

cm = confusion_matrix(true_devs, pred_devs)

idx_to_device = {v: k for k, v in class_to_idx.items()}
device_names = [idx_to_device[i] for i in sorted(idx_to_device.keys())]

#calls function witten above
plot_confusion_matrix(cm, device_names, title=f"Device Confusion Matrix")

**8b) Plot Receiver-Operator Curve (ROC)**

An ROC curve plots the rate of false positives versus the rate of true positives.  In our case, we just have one (FPR,TPR) pair and the (0,0) - > everything is predicted 0 and (1,1) points -> everything predicted 1.

In [ ]:
def plot_roc_curve(y_test, y_pred, class_labels):
  
  n_classes = len(np.unique(y_test))
  y_test = label_binarize(y_test, classes=np.arange(n_classes))
  y_pred = label_binarize(y_pred, classes=np.arange(n_classes))

  # Compute ROC curve and ROC area for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
  
  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  # First aggregate all false positive rates
  all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

  # Then interpolate all ROC curves at this points
  mean_tpr = np.zeros_like(all_fpr)
  for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

  # Finally average it and compute AUC
  mean_tpr /= n_classes

  #fpr["macro"] = all_fpr
  #tpr["macro"] = mean_tpr
  #roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

  # Plot all ROC curves
  plt.figure(figsize=(7,5))
  lw = 2
  plt.plot(fpr["micro"], tpr["micro"],
    label="Average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
    color="deeppink", linestyle=":", linewidth=4,)

  #plt.plot(fpr["macro"], tpr["macro"],
    #label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    #color="navy", linestyle=":", linewidth=4,)

  colors = cycle(["aqua", "darkorange", "darkgreen", "yellow", "blue"])
  for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
        label=" Class = {0} (area = {1:0.2f})".format(device_names[i], roc_auc[i]),)

  plt.plot([0, 1], [0, 1], "k--", lw=lw)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("Receiver Operating Characteristic (ROC) curve")
  plt.legend()

In [ ]:
plot_roc_curve(df_results_val1['true_device'].values,df_results_val1['pred_device'].values, device_names)

## 9. Improve model performace

**9a) Let's explore a different optimizer and learning rate.**

In [ ]:
# Define batch size for data loading
batch_size = 64
torch.set_num_threads(os.cpu_count())
num_workers = os.cpu_count()

# Create DataLoader instances for efficient batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load the ResNet50 model
model = models.resnet50(weights=None)
# Modify the last layer for our classification task
# Changing to 4 classes as we have 4 devices to classify it into
model.fc = nn.Linear(model.fc.in_features, num_classes)
# Transfer model to device (GPU or CPU)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

#set the optimier and learning rate (lr)
optimizer = optim.Adam(model.parameters(), lr=7e-6, weight_decay=1e-3)   #<- this is what we are changing!!

start = time.time()
train_model(model=model, train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            num_epochs=24)
print("Training time: ", time.time() - start)

In [ ]:
# --------------------- VAL ---------------------
all_preds = []
all_labels = []
all_dm_severity = []
print("Evaluating on val set...")
if len(val_df) > 0:
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels, dm_severity in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_dm_severity.extend(dm_severity)

    val_loss /= val_total
    val_acc = val_correct / val_total
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


    # Create a DataFrame to store results
df_results_val2 = pd.DataFrame({
    'true_device': all_labels,
    'pred_device': all_preds,
    'dm_severity': all_dm_severity
})


In [ ]:
# Generate confusion matrices with confusion_matrix function from scikit-learn
true_devs = df_results_val2['true_device'].values
pred_devs = df_results_val2['pred_device'].values
cm = confusion_matrix(true_devs, pred_devs)

idx_to_device = {v: k for k, v in class_to_idx.items()}
device_names = [idx_to_device[i] for i in sorted(idx_to_device.keys())]

#calls function witten above
plot_confusion_matrix(cm, device_names, title=f"Device Confusion Matrix")

In [ ]:
#plot the ROC curve
plot_roc_curve(df_results_val2['true_device'].values,df_results_val2['pred_device'].values, device_names)

**9b) Second the weight initialiation will be changed by using the pretrained weights (ImageNet) for the ResNet50.**

In [ ]:
# Define batch size for data loading
batch_size = 64
torch.set_num_threads(os.cpu_count())
num_workers = os.cpu_count()

# Create DataLoader instances for efficient batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load the ResNet50 model
model = models.resnet50(weights = ResNet50_Weights.IMAGENET1K_V2) #<- this is what we are changing!!
# Modify the last layer for our classification task
# Changing to 4 classes as we have 4 devices to classify it into
model.fc = nn.Linear(model.fc.in_features, num_classes)
# Transfer model to device (GPU or CPU)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

#set the optimier and learning rate (lr)
optimizer = optim.Adam(model.parameters(), lr=7e-6, weight_decay=1e-3)

start = time.time()
train_model(model=model, train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            num_epochs=30)
print("Training time: ", time.time() - start)



In [ ]:
# --------------------- VAL ---------------------
all_preds = []
all_labels = []
all_dm_severity = []
print("Evaluating on val set...")
if len(val_df) > 0:
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels, dm_severity in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_dm_severity.extend(dm_severity)

    val_loss /= val_total
    val_acc = val_correct / val_total
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


    # Create a DataFrame to store results
df_results_val3 = pd.DataFrame({
    'true_device': all_labels,
    'pred_device': all_preds,
    'dm_severity': all_dm_severity
})

In [ ]:
# Generate confusion matrices with confusion_matrix function from scikit-learn
true_devs = df_results_val3['true_device'].values
pred_devs = df_results_val3['pred_device'].values
cm = confusion_matrix(true_devs, pred_devs)

idx_to_device = {v: k for k, v in class_to_idx.items()}
device_names = [idx_to_device[i] for i in sorted(idx_to_device.keys())]

#calls function witten above
plot_confusion_matrix(cm, device_names, title=f"Device Confusion Matrix")

In [ ]:
#plot the ROC curve
plot_roc_curve(df_results_val3['true_device'].values,df_results_val3['pred_device'].values, device_names)

## 10. Choose a model to evaluate on the test data

**The model has not seen these images so this is the least biased way to evaluate a model's perfroamce.  But you can only run one model on your test set, so choose carefully.**

In [ ]:
# Evaluate on the test set

all_preds = []
all_labels = []
all_dm_severity = []
print("Evaluating on test set...")
if len(test_df) > 0:
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels, dm_severity in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_dm_severity.extend(dm_severity)

    test_loss /= test_total
    test_acc = test_correct / test_total
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    # Create a DataFrame to store results
df_results_test = pd.DataFrame({
    'true_device': all_labels,
    'pred_device': all_preds,
    'dm_severity': all_dm_severity
})

In [ ]:
# Generate confusion matrices with confusion_matrix function from scikit-learn
true_devs = df_results_test['true_device'].values
pred_devs = df_results_test['pred_device'].values
cm = confusion_matrix(true_devs, pred_devs)

idx_to_device = {v: k for k, v in class_to_idx.items()}
device_names = [idx_to_device[i] for i in sorted(idx_to_device.keys())]

#calls function witten above
plot_confusion_matrix(cm, device_names, title=f"Device Confusion Matrix")

In [ ]:
#plot the ROC curve
plot_roc_curve(df_results_test['true_device'].values,df_results_test['pred_device'].values, device_names)

In [ ]:
print("Total time: ", time.time() - start_total)

## 11. Sandbox: Explore hyperparameter settings.

**This code was set up for you to explore what happens when you change hyperparameters.**

**Things you could try:**

1. Change data augmentation parameters.
2. Change batch size (warning you may run into memory constraints)
3. Change the initialization seed to any number
4. Change the initialization of the weights (model = models.resnet50(weights = ResNet50_Weights.IMAGENET1K_V2))
5. Change the optimizer and/or learning rate (https://pytorch.org/docs/main/optim.html)
6. Change the number of epocs (you want to see model convergence, training curve is flat at end)


In [ ]:
# Defining transformation of image
# Augmenting training data to have horizontal flips and rotations
train_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally
    T.RandomRotation(degrees=15),  # Randomly rotate images
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])
val_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Loading datasets into memory for faster training
train_dataset = AIREADIDataset_grayscalecrop(train_df, train_transform, preload=True)
val_dataset   = AIREADIDataset_grayscalecrop(val_df, val_transform, preload=True)
test_dataset  = AIREADIDataset_grayscalecrop(test_df, val_transform, preload=True)

# Define batch size for data loading
batch_size = 64
torch.set_num_threads(os.cpu_count())
num_workers = os.cpu_count()

# Create DataLoader instances for efficient batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, worker_init_fn=lambda _: np.random.seed(42))

# Define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

#this will make the initializtion the same each time it is run for the same setting of parameters

set_seed(12)

# Load the ResNet50 model
model = models.resnet50(weights=None)
# Modify the last layer for our classification task
# Changing to 4 classes as we have 4 devices to classify it into
model.fc = nn.Linear(model.fc.in_features, num_classes)
# Transfer model to device (GPU or CPU)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

#set the optimier and learning rate (lr)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=.5)

start = time.time()
train_model(model=model, train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            num_epochs=24)
print("Training time: ", time.time() - start)

In [ ]:
# clean GPU memory
import torch

torch.cuda.empty_cache()
torch.cuda.ipc_collect()